In [ ]:
#%pip install streamlit

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached streamlit-1.44.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached protobuf-5.29.4-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached pyarrow-19.0.1-cp313-cp313-win_amd64.whl.metadata (3.4 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached typing_extensions-4.13.0-py3-none-any.whl.metadata (3.0 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadat


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#%pip install plotly

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: plotly in c:\users\lapep\appdata\local\programs\python\python313\lib\site-packages (6.0.1)




[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
! npm install localtunnel

npm notice
npm notice New major version of npm available! 10.9.2 -> 11.2.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.2.0
npm notice To update run: npm install -g npm@11.2.0
npm notice



added 22 packages in 7s

3 packages are looking for funding
  run `npm fund` for details


In [ ]:
%%writefile app.py
import streamlit as st
import plotly.express as px
import pandas as pd

st.set_page_config(page_title="Amsterdam Airbnb", layout="wide", page_icon="🏠")

# Cargar los datos
@st.cache_resource
def load_data():
    df = pd.read_csv("Datos_limpios_Amsterdam.csv")
    
    numeric_df = df.select_dtypes(include=['float64', 'int64'])
    numeric_cols = numeric_df.columns.tolist()
    text_df = df.select_dtypes(include=['object', 'bool'])
    text_cols = text_df.columns.tolist()
    
    # Columnas importantes (puedes ajustar según tus datos)
    category_col = 'host_is_superhost' if 'host_is_superhost' in df.columns else text_cols[0]
    unique_categories = df[category_col].dropna().unique()

    return df, numeric_cols, text_cols, unique_categories, numeric_df

# Cargar datos
df, numeric_cols, text_cols, unique_categories, numeric_df = load_data()

# Sidebar
st.sidebar.title("🎛️ Panel de control")
st.sidebar.markdown("Navega por diferentes visualizaciones")
view = st.sidebar.radio("Selecciona una vista:", 
                        ["📈 Precios por categoría", 
                         "📊 Dispersión de variables", 
                         "🥧 Distribución categórica", 
                         "📊 Comparativa por categoría", 
                         "📦 Análisis adicionales"])

# Título principal
st.title("🏠 Amsterdam Airbnb Dashboard")
st.markdown("Visualización de datos sobre alojamientos, precios y comportamiento de anfitriones en Amsterdam.")

# VISTA 1: LINE PLOT
if view == "📈 Precios por categoría":
    st.header("📈 Evolución de precios por categoría")

    category_col = st.sidebar.selectbox("Categoría (e.g., superhost, tipo de habitación)", text_cols)
    selected_category = st.sidebar.selectbox("Selecciona valor:", df[category_col].dropna().unique())
    y_vars = st.sidebar.multiselect("Variables numéricas a mostrar", numeric_cols, default=["price"])
    
    filtered = df[df[category_col] == selected_category]
    if not filtered.empty and y_vars:
        st.write(f"Mostrando datos para **{selected_category}** en **{category_col}**")
        fig = px.line(filtered[y_vars], x=filtered.index, y=y_vars, title="Evolución de variables seleccionadas")
        st.plotly_chart(fig)
    else:
        st.warning("Selecciona al menos una variable y asegúrate de que haya datos.")

# VISTA 2: SCATTER PLOT
elif view == "📊 Dispersión de variables":
    st.header("📊 Relación entre variables numéricas")

    x_var = st.sidebar.selectbox("Eje X", numeric_cols)
    y_var = st.sidebar.selectbox("Eje Y", numeric_cols)
    color_cat = st.sidebar.selectbox("Color por categoría", text_cols)

    fig = px.scatter(df, x=x_var, y=y_var, color=df[color_cat], 
                     title=f"Relación entre {x_var} y {y_var} por {color_cat}",
                     size='price' if 'price' in numeric_cols else None)
    st.plotly_chart(fig)

# VISTA 3: PIE PLOT
elif view == "🥧 Distribución categórica":
    st.header("🥧 Distribución por categoría")

    cat_col = st.sidebar.selectbox("Variable Categórica", text_cols)
    num_col = st.sidebar.selectbox("Variable Numérica (peso)", numeric_cols)

    fig = px.pie(df, names=cat_col, values=num_col, 
                 title=f"Distribución de {cat_col} basada en {num_col}")
    st.plotly_chart(fig)

# VISTA 4: BAR PLOT
elif view == "📊 Comparativa por categoría":
    st.header("📊 Comparativa de precios u otras variables")

    cat_col = st.sidebar.selectbox("Variable Categórica", text_cols)
    num_col = st.sidebar.selectbox("Variable Numérica", numeric_cols)

    grouped = df.groupby(cat_col)[num_col].mean().reset_index()
    fig = px.bar(grouped, x=cat_col, y=num_col, color=cat_col,
                 title=f"Promedio de {num_col} por {cat_col}")
    fig.update_layout(xaxis={'categoryorder':'total descending'})
    st.plotly_chart(fig)

# VISTA 5: EXTRAS
elif view == "📦 Análisis adicionales":
    st.header("📦 Histogramas y Boxplots")

    col1, col2 = st.columns(2)

    with col1:
        st.subheader("Histograma")
        hist_col = st.selectbox("Variable para Histograma", numeric_cols, key='hist')
        fig_hist = px.histogram(df, x=hist_col, color='host_is_superhost' if 'host_is_superhost' in df.columns else None)
        st.plotly_chart(fig_hist)

    with col2:
        st.subheader("Boxplot")
        box_col = st.selectbox("Variable para Boxplot", numeric_cols, key='box')
        box_group = st.selectbox("Agrupar por", text_cols, key='group')
        fig_box = px.box(df, x=box_group, y=box_col, color=box_group)
        st.plotly_chart(fig_box)
        

Overwriting app.py
